In [49]:
import numpy as np

n_days = 7 * 7  # 8 weeks
days = np.arange(n_days)

# Weekend: days 6 and 7 of each week
is_weekend = np.array([(d % 7 in [5, 6]) for d in days], dtype=int)

# Variant B in last week
is_variant_b = np.array([1 if d >= 6*7 else 0 for d in days], dtype=int)

# Observations: fewer on weekends
observations = np.where(is_weekend, 
                        np.random.poisson(100, size=n_days), 
                        np.random.poisson(500, size=n_days))

# Conversion rates: 0.05 normally, 0.06 for variant B
p = 0.05 + 0.01 * is_variant_b
successes = np.random.binomial(observations, p)


In [50]:
import pymc as pm
import numpy as np
from scipy.special import logit

# --- Controlled Sequential A/B Test ---

# Baseline conversion rate (Baseline conversion rate for the reference group: Variant A on weekdays)
assumed_base_rate = 0.05  # 5%
base_rate_uncertainty_pp = 0.015

base_logit_mu = logit(assumed_base_rate)

base_logit_sigma = (
    logit(assumed_base_rate + base_rate_uncertainty_pp) -
    logit(assumed_base_rate - base_rate_uncertainty_pp)
) / 4

# Weekend effect: 50% of weekday conversion, range 25%–75%
weekend_multiplier_mu = np.log(0.5)
weekend_multiplier_sigma = (np.log(0.75) - np.log(0.25)) / 4

# Variant B expected uplift: ±2.5 percentage points around 5%
uplift_range_pp = 0.025

uplift_logit_sigma = (
    logit(assumed_base_rate + uplift_range_pp) -
    logit(assumed_base_rate - uplift_range_pp)
) / 4

# --- Probabilistic Model ---

with pm.Model() as conversion_rate_model:

    # Baseline conversion (Variant A, weekday)
    logit_conversion_baseline = pm.Normal(
        "logit_conversion_baseline",
        mu=base_logit_mu,
        sigma=base_logit_sigma
    )

    # Weekend adjustment (log-scale reduction in conversion)
    logit_weekend_adjustment = pm.Normal(
        "logit_weekend_adjustment",
        mu=weekend_multiplier_mu,
        sigma=weekend_multiplier_sigma
    )

    # Uplift from Variant B vs Variant A (in log-odds)
    logit_uplift_variant_b = pm.Normal(
        "logit_uplift_variant_b",
        mu=0,
        sigma=uplift_logit_sigma
    )

    # Linear predictor
    logit_conversion = (
        logit_conversion_baseline
        + logit_weekend_adjustment * is_weekend
        + logit_uplift_variant_b * is_variant_b
    )

    conversion_rate = pm.math.sigmoid(logit_conversion)

    # Observed conversion data
    observed_conversions = pm.Binomial(
        "observed_conversions",
        n=observations,
        p=conversion_rate,
        observed=successes
    )

    # Inference
    trace = pm.sample(1000, tune=1000, target_accept=0.95)



Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [logit_conversion_baseline, logit_weekend_adjustment, logit_uplift_variant_b]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 0 seconds.


In [29]:
pm.summary(trace)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
logit_conversion_a_weekday,-3.019,0.072,-3.154,-2.884,0.001,0.001,2793.0,2946.0,1.0
logit_weekend_conversion_adjustment,-0.651,0.233,-1.101,-0.226,0.004,0.004,2759.0,2273.0,1.0
logit_uplift_variant_b,0.236,0.113,0.027,0.452,0.002,0.002,2645.0,2769.0,1.0


In [51]:
import arviz as az
from scipy.special import expit
import numpy as np

# Summary from the trace with new variable names
summary = az.summary(trace, var_names=[
    "logit_conversion_baseline",
    "logit_weekend_adjustment",
    "logit_uplift_variant_b"
])
print(summary)

# Extract samples
logit_base = trace.posterior["logit_conversion_baseline"].values.flatten()
logit_uplift = trace.posterior["logit_uplift_variant_b"].values.flatten()


# Convert to probabilities
p_base = expit(logit_base)
p_b = expit(logit_base + logit_uplift)
p_weekend = expit(logit_base + logit_weekend)

# Percentage point differences
pp_lift = (p_b - p_base) * 100


# Interpretation prints
print(f"\n📊 Base conversion rate (Variant A weekday): {p_base.mean():.2f}%")
print(f"📈 Conversion rate with Variant B: {p_b.mean():.2f}%")
print(f"🔼 Uplift from Variant B: {pp_lift.mean():.2f} pp (95% HDI: {np.percentile(pp_lift, [2.5, 97.5])})")

# Probability that B is better than A
prob_b_better = np.mean(p_b > p_base)
print(f"✅ Probability that Variant B is better than A: {prob_b_better:.1%}")

# Regret if choosing Variant B and it's actually worse
regret_if_choosing_b = np.where(p_base > p_b, p_base - p_b, 0) * 100  # in percentage points
print(f"⚠️  Expected regret (if choosing Variant B): {regret_if_choosing_b.mean():.2f} pp")
print(f"   95% HDI for regret: {np.percentile(regret_if_choosing_b, [2.5, 97.5])}")



                            mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  \
logit_conversion_baseline -2.908  0.035  -2.974   -2.843      0.001    0.001   
logit_weekend_adjustment  -0.227  0.124  -0.475   -0.012      0.002    0.002   
logit_uplift_variant_b     0.153  0.085  -0.007    0.309      0.001    0.001   

                           ess_bulk  ess_tail  r_hat  
logit_conversion_baseline    2831.0    2475.0    1.0  
logit_weekend_adjustment     3402.0    2675.0    1.0  
logit_uplift_variant_b       3407.0    2746.0    1.0  

📊 Base conversion rate (Variant A weekday): 0.05%
📈 Conversion rate with Variant B: 0.06%
🔼 Uplift from Variant B: 0.82 pp (95% HDI: [-0.09741036  1.73560889])
✅ Probability that Variant B is better than A: 96.2%
⚠️  Expected regret (if choosing Variant B): 0.01 pp
   95% HDI for regret: [0.         0.09741036]


In [7]:
from scipy.special import expit

intercept = -2.995
beta_variant_b = 0.406

p_base = expit(intercept)
p_b = expit(intercept + beta_variant_b)
lift = p_b - p_base

print(f"Lift: {lift:.4f} ({lift * 100:.2f} percentage points)")


Lift: 0.0222 (2.22 percentage points)
